## Тренировочные упражнения по теме _"Аналитические функции"_

### Описание задачи
Ноутбук содержит решение тренировочных задач с использованием аналитических (оконных) функций.

### Дано: *5 таблиц*

#### 1. Таблица *titles*:

|Название атрибута |	Тип атрибута | Смысловое значение | 
|:---|:---:|:---|
|id				|int	|ID сериала/фильма |
|genres			|text	|Жанр |
|original_title	|text	|Оригинальное название|
|popular_title	|text	|Популярное название|
|is_adult		|bool	|Метка 18+|
|type			|text	|Вид _(i.e. фильм, сериал, короткометражка)_|
|run_time		|int	|Продолжительность|
|year_of_start	|int	|Год первого выпуска |
|year_of_end	|int	|Год последнего выпуска|

#### 2. Таблица *episodes*:

|Название атрибута |	Тип атрибута | Смысловое значение | 
|:---|:---:|:---|
|title_id		|int	|ID серии |
|parent_id		|int	|ID сериала, которому принадлежит серия |
|season			|int	|Номер сезона|
|episode		|int	|Номер серии|

#### 3. Таблица *ratings*:

|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|title_id		|int	|ID сериала/фильма |
|avg_rating		|numeric|Средний рейтинг |
|vote_cnt		|int	|Число отданных голосов|

#### 4. Таблица *principals*:

|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|title_id		|int	|ID сериала/фильма |
|ordering		|int	|Номер в списке каста  |
|person_id		|int	|ID члена каста |
|category		|text	|Категория члена каста *(i.e. actor, writer etc)* |
|job			|text	|Должность члена каста *(i.e. writer, producer etc)*|
|characters		|text	|Имя сыгранного персонажа |

#### 5. Таблица *persons*:
|Название атрибута	|Тип атрибута	|Смысловое значение|
|:---|:---:|:---|
|id				|int	|ID персоналии |
|name			|text	|Имя персоналии  |
|professions	|text	|Должность персоналии |
|year_of_birth	|int	|Год рождения |
|year_of_death	|int	|Год смерти|


### Тренировочная задача 1: 
Напишите SQL-запрос, который выведет:  
— название сериала *(назовите поле serial_name)*;  
— название эпизода *(назовите поле episode_name)*;  
— номер сезона;  
— номер эпизода в сезоне;  
— сквозной номер *(назовите поле rn)*.
Результат отсортируйте по возрастанию *serial_name, episode_name, season, episode, rn*.

```
SELECT out.original_title AS serial_name,
       (SELECT inr.original_title
          FROM titles AS inr
         WHERE ep.title_id = inr.id) AS episode_name,
       ep.season,
       ep.episode,
       ROW_NUMBER() OVER(PARTITION BY out.original_title ORDER BY ep.season, ep.episode) AS rn
  FROM episodes AS ep 
  JOIN titles AS out ON ep.parent_id = out.id
 ORDER BY 1,2,3,4,5
 
```
**Итоговый вывод**: *27 439 rows*

|serial_name	|episode_name	|season	|episode	|rn|
|:---|:---|:---|:---|:---|
|1000-lb Best Friends	|Beauty and the Beach	|1	|5	|5|
|1000-lb Best Friends	|Carry on Weighward Son	|2	|3	|11|
|1000-lb Best Friends	|Getting Her Sexy Back	|1	|4	|4|
|1000-lb Best Friends	|Hurry Up and Weight	|2	|8	|16|
|1000-lb Best Friends	|Large and in Charge	|1	|2	|2|
|...	|...	|...	|...	|...|
|Zaina Juliette & Friends	|Zaina on Las Vegas Rocks Radio Show	|5	|4	|47|
|Zaina Juliette & Friends	|Zaina on Las Vegas Rocks Radio Show Pt 2	|5	|5	|48|
|Zaina Juliette & Friends	|Zaina's Tribute to Classic Entertainers 2	|6	|4	|58|
|Zaina Juliette & Friends	|Zaina's Tribute to Classic Entertainers Part 3	|6	|5	|59|
|Zuuzuu's Friends	|Candy and the Curd	|1	|1	|1|

***

### Тренировочная задача 2: 
Напишите SQL-запрос, который выведет количество умерших людей из таблицы *persons* для каждого года, начиная с 2000-го, накопленным итогом.

Например, для 2012 года будет указано количество людей, которые ушли из жизни с 2000 по 2012 год.

Поля к выводу: год *(year_of_death)*, кумулятивная сумма на этот год *(cume_cnt)*. Отсортируйте результат по их возрастанию.

```
SELECT gr.year_of_death,
       SUM(gr.cnt) OVER (ORDER BY year_of_death) AS cume_cnt
  FROM (
        SELECT p.year_of_death,
               COUNT(p.id) AS cnt
          FROM persons AS p
         WHERE p.year_of_death >= 2000
         GROUP BY p.year_of_death
        ) AS gr;       
 
```
**Итоговый вывод**: *2 rows*

|year_of_death	|cume_cnt|
|---:|:---:|
|2000	|92|
|2001	|199|
|2002	|316|
|2003	|445|
|2004	|564|
|...	|...|
|2010	|1277|
|2011	|1430|
|2012	|1597|
|2013	|1733|
|2014	|1914|
|...	|...|
|2020	|3026|
|2021	|3253|
|2022	|3468|
|2023	|3653|
|2024	|3720|

***

### Тренировочная задача 3: 

Напишите SQL-запрос, который выведет:

— *year_of_start* из таблицы *titles*;

— количество произведений по годам накопленным итогом, набравших максимальный средний рейтинг, равный 10.

Для выполнения запроса используйте таблицы *titles* и *ratings*. Поля к выводу: год *(year_of_start)*, кумулятивная сумма *(cume_cnt)*. Отсортируйте результат по их возрастанию.

```
  SELECT year_of_start,
         SUM(cnt) OVER (ORDER BY year_of_start) AS cume_cnt
    FROM (SELECT titles.year_of_start,
                 COUNT(titles.id) AS cnt
            FROM titles
           WHERE titles.id IN (SELECT ratings.title_id
                                 FROM ratings
                                WHERE ratings.avg_rating = 10)
           GROUP BY titles.year_of_start) AS start_data
   ORDER BY 1, 2;
 
```
**Итоговый вывод**: *61 rows*

|year_of_start	|cume_cnt|
|:---|:---|
|1952	|1|
|1955	|4|
|1957	|5|
|1958	|9|
|1959	|10|
|...	|...|
|2020	|4934|
|2021	|5275|
|2022	|5807|
|2023	|6400|
|2024	|6669|

***

### Тренировочная задача 4: 
Напишите SQL-запрос, который выведет:

— название сериала (назовите поле *serial_name*);

— название эпизода (назовите поле *episode_name*);

— номер сезона;

— номер эпизода в сезоне;

— максимальный рейтинг эпизодов до этой серии включительно (назовите поле *max_rating*);

Серии с пустыми значениями сезона и эпизода показывать не нужно.

Результат отсортируйте по всем колонкам в том порядке, в котором они расположены выше (сортировка в порядке возрастания).

```
  SELECT titles.original_title AS serial_name,
         (SELECT t.original_title
            FROM titles AS t
           WHERE t.id = episodes.title_id) AS episode_name,
         episodes.season,
         episodes.episode,
         MAX(ratings.avg_rating) OVER (PARTITION BY episodes.parent_id ORDER BY season, episode) AS max_rating
    FROM episodes
    LEFT JOIN ratings
      ON episodes.title_id = ratings.title_id
    JOIN titles
      ON episodes.parent_id = titles.id
   WHERE episodes.season       IS NOT NULL 
     AND episodes.episode      IS NOT NULL 
     AND ratings.avg_rating    IS NOT NULL 
   ORDER BY 1,2,3,4,5;
 
```
**Итоговый вывод**: *5041 rows*

|serial_name	|episode_name	|season	|episode	|max_rating|
|:---|:---|:---:|:---:|:---:|
|1000-lb Best Friends	|Beauty and the Beach	|1	|5	|6.7|
|1000-lb Best Friends	|Getting Her Sexy Back	|1	|4	|6.7|
|1000-lb Best Friends	|Large and in Charge	|1	|2	|6.7|
|1000-lb Best Friends	|Ride or Die from Jr. High	|1	|8	|6.7|
|1000-lb Best Friends	|Supersized Besties	|1	|1	|6.7|
|...	|...	|...	|...	|...|
|World Class Championship Wrestling: Star Wars	|WCCW Wrestling Star Wars 1981	|1	|1	|5.6|
|Wrigley Friends	|Big Red's Eye Exam	|1	|2	|9.2|
|Wrigley Friends	S|pearmint's Art Exhibit	|1	|4	|9.7|
|Year Friends	|January	|1	|1	|7.6|
|Your Friends Will Never Believe You	|YFWNBY: Jennifer Love Hewitt	|1	|5	|9.8|


***

### Тренировочная задача 5: 
Напишите SQL-запрос, который выведет:

— название сериала  (назовите поле *serial_name*);

— название эпизода (назовите поле *episode_name*);

— номер сезона;

— номер эпизода в сезоне;

— количество голосов, отданных за сериал до текущего эпизода включительно (назовите поле *vote_cnt*);

Результат отсортируйте по возрастанию *serial_name, episode_name, season, episode, vote_cnt*.

```
  SELECT titles.original_title AS serial_name,
         (SELECT t.original_title
            FROM titles AS t
           WHERE t.id = episodes.title_id) AS episode_name,
         episodes.season,
         episodes.episode,
         SUM(ratings.vote_cnt) OVER (PARTITION BY episodes.parent_id ORDER BY episodes.season, episodes.episode) AS vote_cnt
    FROM episodes
    JOIN titles  ON episodes.parent_id = titles.id
    JOIN ratings ON episodes.title_id = ratings.title_id
   ORDER BY 1,2,3,4,5;   
 
```
**Итоговый вывод**: *5164 rows*

|serial_name	|episode_name	|season	|episode	|vote_cnt|
|:---|:---|:---:|:---:|:---:|
|1000-lb Best Friends	|Beauty and the Beach	|1	|5	|35|
|1000-lb Best Friends	|Getting Her Sexy Back	|1	|4	|29|
|1000-lb Best Friends	|Large and in Charge	|1	|2	|18|
|1000-lb Best Friends	|Ride or Die from Jr. High	|1	|8	|40|
|1000-lb Best Friends	|Supersized Besties	|1	|1	|11|
|...	|...	|...	|...	|...|
|My Friends Tigger & Pooh	|Super-Sized Darby/Piglet's Lightning Frightening	|1	|13	|209|
|My Friends Tigger & Pooh	|Symphony for a Rabbit/Tigger Goes Snowflakey|1	|21	|340|
|My Friends Tigger & Pooh	|Tigger Goes for the Jagular/Rabbit's New Roomie	|1	|10|165|
|My Friends Tigger & Pooh	|You Ain't Just Whistlin', Tigger/Piglet's Hole Problem	|1	|9	|152|
|My Little Pony: Friendship Is Magic	|2, 4, 6, Greaaat	|9	|15	|123569|
|...	|...|	...	|...	|...|
|World Class Championship Wrestling: Star Wars	|WCCW Wrestling Star Wars 1981	|1	|1	|8|
|Wrigley Friends	|Big Red's Eye Exam	|1	|2	|11|
|Wrigley Friends	|Spearmint's Art Exhibit	|1	|4	|17|
|Year Friends	|January	|1	|1	|5|
|Your Friends Will Never Believe You	|YFWNBY: Jennifer Love Hewitt	|1	|5	|6|


***

### Тренировочная задача 6: 
Напишите SQL-запрос, который выведет:

— название сериала (назовите поле *serial_name*);

— название эпизода (назовите поле *episode_name*);

— номер сезона;

— номер эпизода в сезоне;

— расчетное поле: если предыдущий эпизод получил оценку хуже — better, если лучше — worse, если такую же — the same. Для первого эпизода значение принимается NULL. Назовите поле *mark*.

Результат отсортируйте по возрастанию *serial_name, season, episode, episode_name*.

```
WITH cte AS (SELECT titles.original_title AS serial_name,
                    (SELECT t.original_title 
                       FROM titles AS t
                      WHERE t.id = episodes.title_id) AS episode_name,
                    episodes.season,
                    episodes.episode,
                    ratings.avg_rating,
                    LAG(ratings.avg_rating) OVER (PARTITION BY episodes.parent_id ORDER BY episodes.season, episodes.episode)
               FROM episodes
               JOIN titles ON episodes.parent_id = titles.id
               JOIN ratings ON episodes.title_id = ratings.title_id)
  SELECT serial_name,
         episode_name,
         season, 
         episode,
         (CASE 
            WHEN lag is null      THEN null
            WHEN avg_rating > lag THEN 'better' 
            WHEN avg_rating < lag THEN 'worse'
            WHEN avg_rating = lag THEN 'the same'
          END) AS mark
    FROM cte
   ORDER BY 1,3,4,2;  

 
```
**Итоговый вывод**: *5164 rows*

|serial_name	|episode_name	|season	|episode	|mark|
|:---|:---|:---:|:---:|:---:|
|1000-lb Best Friends	|Supersized Besties	|1	|1	|None|
|1000-lb Best Friends	|Large and in Charge	|1	|2	|worse
|1000-lb Best Friends	|We're Not a Joke	|1	|3	|better|
|1000-lb Best Friends	|Getting Her Sexy Back	|1	|4	|worse|
|1000-lb Best Friends	|Beauty and the Beach	|1	|5	|better|
|...|	...	|...	|...	|...|
|My Friends Tigger & Pooh	|Darby Goes Woozle Sleuthin'/How the Tigger Lost His Stripe	|2	|1	|worse|
|My Friends Tigger & Pooh	|Pooh's Cookie Tree/Lumpy Joins In	|2	|3	|better|
|My Friends Tigger & Pooh	|Rabbit's Sound of Silence/Eager Beaver|	2	|5	|better|
|My Friends Tigger & Pooh	|No More Honey for Pooh/Stuck Be a Piglet	|2	|7|	worse|
|My Little Pony: Friendship Is Magic	|Friendship Is Magic - Part 1 (Mare in the Moon)	|1	|1	|None|
|...|	...	|...|	...|	...|
|World Class Championship Wrestling: Star Wars	|WCCW Wrestling Star Wars 1981	|1	|1	|None|
|Wrigley Friends	|Big Red's Eye Exam	|1	|2	|None|
|Wrigley Friends	|Spearmint's Art Exhibit	|1	|4	|better|
|Year Friends	|January	|1	|1	|None|
|Your Friends Will Never Believe You	|YFWNBY: Jennifer Love Hewitt	|1	|5	|None|


***

### Тренировочная задача 7: 
Напишите SQL-запрос, который выведет:

— название сериала (назовите поле *serial_name*);

— название эпизода (назовите поле *episode_name*);

— номер сезона;

— номер эпизода в сезоне;

— ранг эпизода с пропусками по убыванию среднего рейтинга (назовите поле *rnk*);

Результат отсортируйте по возрастанию *serial_name, season, episode, episode_name*.
```
WITH cte AS (SELECT titles.original_title AS serial_name,
                    (SELECT t.original_title 
                       FROM titles AS t
                      WHERE t.id = episodes.title_id) AS episode_name,
                    episodes.season,
                    episodes.episode,
                    ratings.avg_rating,
                    episodes.parent_id
               FROM episodes
               JOIN titles ON episodes.parent_id = titles.id
               JOIN ratings ON episodes.title_id = ratings.title_id)
  SELECT serial_name,
         episode_name,
         season,
         episode,
         RANK() OVER (PARTITION BY parent_id ORDER BY avg_rating DESC) AS rnk
    FROM cte 
ORDER BY 1,3,4,2;  
 
```
**Итоговый вывод**: *5164 rows*

|serial_name	|episode_name	|season	|episode	|rnk|
|:---|:---|:---:|:---:|:---:|
|1000-lb Best Friends	|Supersized Besties	|1	|1	|2|
|1000-lb Best Friends	|Large and in Charge	|1	|2	|6|
|1000-lb Best Friends	|We're Not a Joke	|1	|3	|2|
|1000-lb Best Friends	|Getting Her Sexy Back	|1	|4	|7|
|1000-lb Best Friends	|Beauty and the Beach	|1	|5	|2|
|...	|...	|...	|...|	...|
|My Friends Tigger & Pooh	|Darby Goes Woozle Sleuthin'/How the Tigger Lost His Stripe|	2	|1	|27|
|My Friends Tigger & Pooh	|Pooh's Cookie Tree/Lumpy Joins In	|2|	3	|5|
|My Friends Tigger & Pooh	|Rabbit's Sound of Silence/Eager Beaver	|2|	5	|3|
|My Friends Tigger & Pooh	|No More Honey for Pooh/Stuck Be a Piglet	|2	|7	|7|
|My Little Pony: Friendship Is Magic	|Friendship Is Magic - Part 1 (Mare in the Moon)	|1	|1|	131|
|...|	...|	...	|...	|...|
|World Class Championship Wrestling: Star Wars	|WCCW Wrestling Star Wars 1981	|1	|1	|1|
|Wrigley Friends	|Big Red's Eye Exam	|1	|2	|2|
|Wrigley Friends	|Spearmint's Art Exhibit|	1|	4|	1|
|Year Friends	|January	|1	|1	|1|
|Your Friends Will Never Believe You	|YFWNBY: Jennifer Love Hewitt	|1|	5	|1|

***

### Тренировочная задача 8: 
Напишите SQL-запрос, который выведет:

— название сериала (назовите поле *serial_name*);

— название эпизода (назовите поле *episode_name*);

— номер сезона;

— номер эпизода в сезоне;

— рейтинг этого эпизода;

— максимальный рейтинг эпизодов среди последних 3 эпизодов, включая текущий (назовите поле *max_rating*);

Результат отсортируйте по возрастанию *serial_name, season, episode, episode_name*.
```
WITH cte AS (SELECT titles.original_title AS serial_name,
                    (SELECT t.original_title 
                       FROM titles AS t
                      WHERE t.id = episodes.title_id) AS episode_name,
                    episodes.season,
                    episodes.episode,
                    ratings.avg_rating,
                    episodes.parent_id
               FROM episodes
               JOIN titles ON episodes.parent_id = titles.id
               JOIN ratings ON episodes.title_id = ratings.title_id)
  SELECT serial_name,
         episode_name,
         season,
         episode,
         avg_rating,
         MAX(avg_rating) OVER (PARTITION BY parent_id ORDER BY season, episode ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS max_rating
    FROM cte     
ORDER BY 1,3,4,2;  
 
```
**Итоговый вывод**: *5164 rows*

|serial_name	|episode_name	|season	|episode	|avg_rating	|max_rating|
|:---|:---|:---:|:---:|:---:|:---:|
|1000-lb Best Friends	|Supersized Besties	|1	|1	|6.7|	6.7|
|1000-lb Best Friends	|Large and in Charge	|1	|2	|5.9|	6.7|
|1000-lb Best Friends	|We're Not a Joke	|1	|3	|6.7|	6.7|
|1000-lb Best Friends	|Getting Her Sexy Back	|1	|4	|5.6|	6.7|
|1000-lb Best Friends	|Beauty and the Beach	|1	|5	|6.7|	6.7|
|...	|...	|...	|...	|...	|...|
|My Friends Tigger & Pooh	|Darby Goes Woozle Sleuthin'/How the Tigger Lost His Stripe	|2	|1	|4.8|	8.3|
|My Friends Tigger & Pooh	|Pooh's Cookie Tree/Lumpy Joins In	|2	|3	|7	|7.1|
|My Friends Tigger & Pooh	|Rabbit's Sound of Silence/Eager Beaver	|2	|5	|8	|8|
|My Friends Tigger & Pooh	|No More Honey for Pooh/Stuck Be a Piglet	|2	|7	|6.3	|8|
|My Little Pony: Friendship Is Magic	|Friendship Is Magic - Part 1 (Mare in the Moon)	|1	|1	|7.6	|7.6|
|...	|...|	...	|...	|...|	...|
|World Class Championship Wrestling: Star Wars	|WCCW Wrestling Star Wars 1981	|1	|1	|5.6	|5.6|
|Wrigley Friends	|Big Red's Eye Exam	|1	|2	|9.2	|9.2|
|Wrigley Friends	|Spearmint's Art Exhibit	|1	|4	|9.7	|9.7|
|Year Friends	|January	|1	|1|	7.6	|7.6|
|Your Friends Will Never Believe You	|YFWNBY: Jennifer Love Hewitt	|1	|5	|9.8|	9.8|



***

### Тренировочная задача 9: 
Напишите SQL-запрос, который выведет количество умерших людей из таблицы persons для каждого года, начиная с 2000-го, накопленным итогом для текущего года и всех последующих лет.

Например, для 2012 года будет указано количество людей, которые ушли из жизни с 2012 по 2024 год.

Поля к выводу: год *(year_of_death)*, кумулятивная сумма на этот год *(cume_cnt)*.

Результат отсортируйте по возрастанию *year_of_death, cume_cnt*.
```
  SELECT year_of_death, 
         SUM(cnt) OVER (ORDER BY year_of_death ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS cume_cnt
    FROM (SELECT year_of_death,
                 COUNT(id) as cnt
            FROM persons
           WHERE year_of_death >= 2000
           GROUP BY year_of_death) AS grouped;
 
```
**Итоговый вывод**: *25 rows*

|year_of_death	|cume_cnt|
|:---|:---:|
|2000	|3720|
|2001	|3628|
|2002	|3521|
|2003	|3404|
|2004	|3275|
|...|	...
|2010	|2578|
|2011	|2443|
|2012	|2290|
|2013	|2123|
|2014	|1987|
|...	|...|
|2020	|903|
|2021	|694|
|2022	|467|
|2023	|252|
|2024	|67|


***

### Тренировочная задача 10: 
Напишите SQL-запрос, который выведет:

— название сериала (назовите поле *serial_name*);

— название эпизода (назовите поле *episode_name*);

— номер сезона;

— номер эпизода в сезоне;

— сквозной номер (назовите поле *num*);

— максимальный рейтинг эпизодов до этой серии включительно (назовите поле *max_rating*);

— количество голосов, отданных за сериал до этой серии включительно (назовите поле *vote_cnt*).
```

 
```
**Итоговый вывод**: *5164 rows*

|serial_name	|episode_name	|season	|episode	|num	|max_rating	|vote_cnt|
|:---|:---|:---:|:---:|:---:|:---:|:---:|
|Rocky and His Friends	|Jet Fuel Formula/Bullwinkle's Ride or Goodbye, Dollink	|1.0	|1.0	|1	|7.6	|98|
|Rocky and His Friends	|Bullseye Bullwinkle or Destination Moose/Squeeze Play or Invitation to the Trance	|1.0	|2.0	|2	|7.7	|168|
|Rocky and His Friends	|The Scrooched Moose/Monitored Moose or The Carbon Copy-Cats	|1.0	|3.0	|3	|7.7	|231|
|Rocky and His Friends	|Rocky's Dilemma or A Squirrel in a Stew/The Submarine Squirrel or 20,000 Leagues Beneath the Sea	|1.0	|4.0	|4	|7.7	|285|
|Rocky and His Friends	|The Bars and Stripes Forever/Hello Out There! or There's No Place Like Space	|1.0	|5.0	|5	|7.7	|336|
|...	|...	|...	|...|	...|	...|	...|
|Star Wars: Tales of the Empire	|Devoted|	1.0	|4.0|	4	|8.4	|5457|
|Star Wars: Tales of the Empire	|Realization	|1.0|	5.0	|5	|8.4	|6767|
|Star Wars: Tales of the Empire	|The Way Out	|1.0|	6.0	|6	|8.4	|8040|
|Cumartesi & Pazar Sürprizi	|Pazar Surprizi Hande Ercel Roportaji	|1.0	|158.0	|1	|10.0|	218|
|Cumartesi & Pazar Sürprizi	|Cannes'da ilgi Odagi Olan Hande Ercel ile Cok Ozel Roportaj	|1.0	|501.0	|2	|10.0	|445|

***